In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/DACON/소득예측")
os.getcwd()

'/content/drive/MyDrive/DACON/소득예측'

In [50]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# OLS

## 데이터 로드

In [4]:
train = pd.read_csv("train_preprocessed.csv")
test = pd.read_csv("test_preprocessed.csv")

display(train.shape, train.head(2))
display(test.shape, test.head(2))

(20000, 34)

,Age,Gender,Working_Week (Yearly),Industry_Status,Hispanic_Origin,Household_Summary,Losses,Associates degree (Academic),Associates degree (Vocational),Bachelors degree,Children,College,Doctorate degree,Elementary (1-4),Elementary (5-6),High Freshman,High Junior,High Senior,High Sophomore,High graduate,Kindergarten,Masters degree,Middle (7-8),Professional degree,Divorced,Married,Married (Armed Force Spouse),Married (Spouse Absent),Separated,Single,Widowed,US_Born,Occu+Employ,Income
0,6,1,4,20,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,13,425
1,3,1,52,6,0,4,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,13,0


(10000, 33)

,Age,Gender,Working_Week (Yearly),Industry_Status,Hispanic_Origin,Household_Summary,Losses,Associates degree (Academic),Associates degree (Vocational),Bachelors degree,Children,College,Doctorate degree,Elementary (1-4),Elementary (5-6),High Freshman,High Junior,High Senior,High Sophomore,High graduate,Kindergarten,Masters degree,Middle (7-8),Professional degree,Divorced,Married,Married (Armed Force Spouse),Married (Spouse Absent),Separated,Single,Widowed,US_Born,Occu+Employ
0,7,1,0,14,0,4,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,14
1,4,1,0,14,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,14


In [5]:
X = train.drop('Income', axis=1)
y = train['Income']

model = sm.OLS(y, sm.add_constant(X))
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 Income   R-squared:                       0.212
Model:                            OLS   Adj. R-squared:                  0.211
Method:                 Least Squares   F-statistic:                     173.1
Date:                Mon, 08 Apr 2024   Prob (F-statistic):               0.00
Time:                        14:44:10   Log-Likelihood:            -1.5706e+05
No. Observations:               20000   AIC:                         3.142e+05
Df Residuals:                   19968   BIC:                         3.144e+05
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const       

> summary상 R^2 상관계수가 매우 낮아 p-value 값이 0.05 이상인 피쳐는 제거하기로 결정

## 컬럼 제거

In [6]:
drop_cols = ['Age', 'Industry_Status', 'Household_Summary', 'Doctorate degree', 'Elementary (1-4)', 'High Freshman', 'High Junior',
             'High Senior', 'High Sophomore', 'Kindergarten', 'Masters degree', 'Middle (7-8)', 'Married (Spouse Absent)',
             'Separated', 'Single', 'Widowed', 'US_Born']

train.drop(columns = drop_cols, axis=1, inplace=True)
test.drop(columns = drop_cols, axis=1, inplace=True)

print(train.shape, test.shape)
display(train.head(2), test.head(2))

(20000, 17) (10000, 16)


,Gender,Working_Week (Yearly),Hispanic_Origin,Losses,Associates degree (Academic),Associates degree (Vocational),Bachelors degree,Children,College,Elementary (5-6),High graduate,Professional degree,Divorced,Married,Married (Armed Force Spouse),Occu+Employ,Income
0,1,4,0,1,0,0,0,0,0,0,0,0,0,1,0,13,425
1,1,52,0,1,0,1,0,0,0,0,0,0,0,0,0,13,0


,Gender,Working_Week (Yearly),Hispanic_Origin,Losses,Associates degree (Academic),Associates degree (Vocational),Bachelors degree,Children,College,Elementary (5-6),High graduate,Professional degree,Divorced,Married,Married (Armed Force Spouse),Occu+Employ
0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,14
1,1,0,8,1,0,0,0,0,0,1,0,0,0,0,0,14


## 컬럼 제거 후 상관계수 재확인

In [7]:
X= train.drop(["Income"],axis =1)
y= train["Income"]

model = sm.OLS(y, sm.add_constant(X))
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 Income   R-squared:                       0.211
Model:                            OLS   Adj. R-squared:                  0.211
Method:                 Least Squares   F-statistic:                     334.3
Date:                Mon, 08 Apr 2024   Prob (F-statistic):               0.00
Time:                        14:44:22   Log-Likelihood:            -1.5707e+05
No. Observations:               20000   AIC:                         3.142e+05
Df Residuals:                   19983   BIC:                         3.143e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const       

> R^2 값이 0.21에서 변화 없음,  
F-statistic 증가,  
AIC 값 변화없음으로  
=> 컬럼 삭제가 유의미하지 않아 표준화를 진행해보기로 함

# Standarization 과 절편 확인

## 데이터 로드

In [8]:
train = pd.read_csv("train_preprocessed.csv")
test = pd.read_csv("test_preprocessed.csv")

display(train.shape, train.head(2))
display(test.shape, test.head(2))

(20000, 34)

,Age,Gender,Working_Week (Yearly),Industry_Status,Hispanic_Origin,Household_Summary,Losses,Associates degree (Academic),Associates degree (Vocational),Bachelors degree,Children,College,Doctorate degree,Elementary (1-4),Elementary (5-6),High Freshman,High Junior,High Senior,High Sophomore,High graduate,Kindergarten,Masters degree,Middle (7-8),Professional degree,Divorced,Married,Married (Armed Force Spouse),Married (Spouse Absent),Separated,Single,Widowed,US_Born,Occu+Employ,Income
0,6,1,4,20,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,13,425
1,3,1,52,6,0,4,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,13,0


(10000, 33)

,Age,Gender,Working_Week (Yearly),Industry_Status,Hispanic_Origin,Household_Summary,Losses,Associates degree (Academic),Associates degree (Vocational),Bachelors degree,Children,College,Doctorate degree,Elementary (1-4),Elementary (5-6),High Freshman,High Junior,High Senior,High Sophomore,High graduate,Kindergarten,Masters degree,Middle (7-8),Professional degree,Divorced,Married,Married (Armed Force Spouse),Married (Spouse Absent),Separated,Single,Widowed,US_Born,Occu+Employ
0,7,1,0,14,0,4,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,14
1,4,1,0,14,8,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,14


## 데이터 분리

In [9]:
# 독립변수와 종속변수 분리
X = train.iloc[:, :-1]
y = train.iloc[:, -1]

# 학습용, 평가용 데이터 생성
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, test_size = 0.3, random_state = 2024)

In [10]:
### 학습용 데이터의 자료형과 형태 확인
print(f'학습용 X 데이터의 자료형 : {type(X_train)}')
print(f'학습용 X 데이터의 모양 : {X_train.shape}')
print('-'*80)
print(f'학습용 y 데이터의 자료형 : {type(y_train)}')
print(f'학습용 y 데이터의 모양 : {y_train.shape}')

학습용 X 데이터의 자료형 : <class 'pandas.core.frame.DataFrame'>
학습용 X 데이터의 모양 : (14000, 33)
--------------------------------------------------------------------------------
학습용 y 데이터의 자료형 : <class 'pandas.core.series.Series'>
학습용 y 데이터의 모양 : (14000,)


In [11]:
### 평가용 데이터의 자료형과 형태 확인
print(f'평가용 X 데이터의 자료형 : {type(X_test)}')
print(f'평가용 X 데이터의 모양 : {X_test.shape}')
print('-'*80)
print(f'평가용 y 데이터의 자료형 : {type(y_test)}')
print(f'평가용 y 데이터의 모양 : {y_test.shape}')

평가용 X 데이터의 자료형 : <class 'pandas.core.frame.DataFrame'>
평가용 X 데이터의 모양 : (6000, 33)
--------------------------------------------------------------------------------
평가용 y 데이터의 자료형 : <class 'pandas.core.series.Series'>
평가용 y 데이터의 모양 : (6000,)


## 표준화 진행

In [15]:
# StandardScaler 객체 생성
scaler = StandardScaler()

# scaling 적용
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

display(X_train_scaled, X_test_scaled)

array([[-0.59144071, -0.95733774,  0.76771659, ..., -0.19758074,
        -2.21577673, -1.77003545],
       [-0.59144071,  1.04456344,  0.76771659, ..., -0.19758074,
         0.45130901,  0.19302492],
       [ 0.48453485, -0.95733774,  0.76771659, ..., -0.19758074,
         0.45130901, -0.59219923],
       ...,
       [-1.12942848,  1.04456344, -1.57181482, ..., -0.19758074,
         0.45130901,  1.56716718],
       [ 1.5605104 , -0.95733774, -1.57181482, ..., -0.19758074,
         0.45130901,  1.56716718],
       [-0.05345293, -0.95733774,  0.76771659, ..., -0.19758074,
         0.45130901,  0.38933096]])

array([[-1.12942848,  1.04456344,  0.76771659, ..., -0.19758074,
         0.45130901,  0.38933096],
       [-1.66741626,  1.04456344, -1.57181482, ..., -0.19758074,
         0.45130901,  0.97824907],
       [-1.12942848, -0.95733774, -1.57181482, ..., -0.19758074,
         0.45130901,  0.97824907],
       ...,
       [-1.66741626, -0.95733774, -1.57181482, ..., -0.19758074,
         0.45130901,  0.97824907],
       [ 1.02252263,  1.04456344, -0.22208516, ..., -0.19758074,
         0.45130901,  0.19302492],
       [-0.59144071, -0.95733774,  0.13784275, ..., -0.19758074,
         0.45130901,  0.585637  ]])

## 선형회귀모델

In [16]:
# 선형 회귀 모델 생성
lr = LinearRegression()
lr_scaled = LinearRegression()

# 학습
lr.fit(X_train, y_train)
lr_scaled.fit(X_train_scaled, y_train)

LinearRegression()

In [30]:
# 기울기, 절편 확인
print("스케일링 진행 X")
print("lr 기울기\n", lr.coef_)
print("\n lr 절편\n ==> ", lr.intercept_)

print('='*80)


print("스케일링(표준화) 진행 O")
print("lr_scaled 기울기\n", lr_scaled.coef_)
print("\n lr_scaled 절편\n ==> ", lr_scaled.intercept_)

스케일링 진행 X
lr 기울기
 [  -0.46169357   99.96698973    8.60474282   -1.55671674  -11.32175446
   -0.88069163  -73.62410601  163.64524221  212.46756318  107.97091267
  -57.19438019   67.17508469    1.30915731  -75.85320469 -158.63579508
  -13.22843198   21.29397938   -2.16369282   -5.816588     62.54954105
 -123.84536976  -25.31470089  -28.54482314 -145.81449393   20.07074463
   23.79770251  129.71363245  -33.39243917  -21.17227291  -69.94100303
  -49.07636449  -13.21773206  -18.10595887]

 lr 절편
 ==>  449.9572678543359
스케일링(표준화) 진행 O
lr_scaled 기울기
 [-4.42605190e-01  4.86255425e+01  1.91085076e+02 -9.26969996e+00
 -1.94037229e+01 -1.79752252e+00 -9.68542254e+00  1.29625758e+15
  1.38746170e+15  2.18952299e+15  2.29344033e+15  2.87830437e+15
  4.96296854e+14  6.40726037e+14  9.19077308e+14  1.20315520e+15
  1.47216246e+15  8.37881381e+14  1.43923434e+15  3.44057636e+15
  4.21027770e+14  1.13576655e+15  1.21072032e+15  5.71522766e+14
  1.32629200e+15  2.40522528e+15  2.93019734e+14  4.58472989

> 표준화를 진행하지 않은 절편이 진행한 절편보다 나은 값이며 둘 간의 큰 차이가 없음  
따라서 절편값이 그나마 낮은 => 스케일링 진행하지 않은 데이터를 선택하여 예측 진행

In [18]:
# 절편값이 높지 않은 모델 (스케일링 진행X) 선택하여 예측
y_pred_train = lr.predict(X_train_scaled)
y_pred_test = lr.predict(X_test_scaled)

display(y_pred_train, y_pred_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([ 463.02434677, 1377.60092717,  694.00030166, ...,  105.61851051,
        403.14729313,  443.56403441])

array([ 293.28479418,  106.51290374,  -95.09261322, ...,  -93.61113064,
        681.27904569, 3277.89787712])

In [19]:
# 성능 평가
train_mse = mean_squared_error(y_train, y_pred_train)
test_mse = mean_squared_error(y_test, y_pred_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print("훈련 데이터셋 MSE:", train_mse)
print("테스트 데이터셋 MSE:", test_mse)
print("훈련 데이터셋 R^2 점수:", r2_train)
print("테스트 데이터셋 R^2 점수:", r2_test)

훈련 데이터셋 MSE: 522748.484531179
테스트 데이터셋 MSE: 607163.9311300302
훈련 데이터셋 R^2 점수: -0.13039554474634896
테스트 데이터셋 R^2 점수: -0.08166527305345417


# Pipeline 모델 선택

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

In [21]:
train = pd.read_csv("train_preprocessed.csv")
test = pd.read_csv("test_preprocessed.csv")

X= train.drop(["Income"],axis =1)
y= train["Income"]

In [22]:
pipeline_lr=Pipeline([("scalar1",StandardScaler()),
                     ("lr_classifier",LinearRegression())])

pipeline_dt=Pipeline([("scalar2",StandardScaler()),
                     ("dt_classifier",DecisionTreeRegressor())])

pipeline_rf=Pipeline([("scalar3",StandardScaler()),
                     ("rf_classifier",RandomForestRegressor())])


pipeline_kn=Pipeline([("scalar4",StandardScaler()),
                     ("rf_classifier",KNeighborsRegressor())])


pipeline_xgb=Pipeline([("scalar5",StandardScaler()),
                     ("rf_classifier",XGBRegressor())])


pipelines = [pipeline_lr, pipeline_dt, pipeline_rf, pipeline_kn, pipeline_xgb]

pipe_dict = {0: "LinearRegression", 1: "DecisionTree", 2: "RandomForest",3: "KNeighbors", 4: "XGBRegressor"}


for pipe in pipelines:
    pipe.fit(X, y)

## 결과

In [23]:
results = []

for i, model in enumerate(pipelines):
    score = cross_val_score(model, X, y, scoring="neg_root_mean_squared_error", cv=10)
    results.append(score)
    print("%s: %f " % (pipe_dict[i], score.mean()))

LinearRegression: -622.715454 
DecisionTree: -825.113392 
RandomForest: -642.546481 
KNeighbors: -661.792639 
XGBRegressor: -629.716718 


> RMSE의 경우 값이 낮을수록 모델의 예측 성능이 우수한 것으로 평가.  
Decision Tree로 진행 해보기로 함

# Decison Tree

In [39]:
train = pd.read_csv("train_preprocessed.csv")
test = pd.read_csv("test_preprocessed.csv")

# 독립변수와 종속변수 분리
X = train.iloc[:, :-1]
y = train.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=2024)

In [40]:
from sklearn.tree import DecisionTreeRegressor

# 객체생성
model = DecisionTreeRegressor()

# 학습
model.fit(X_train, y_train)

# 예측
pred = model.predict(X_test)
pred

array([425.,   0.,   0., ...,   0.,   0.,   0.])

## 모델 평가

In [41]:
mse = mean_squared_error(y_test, pred)
mae = mean_absolute_error(y_test, pred)
r2 = r2_score(y_test, pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R^2 Score:", r2)

Mean Squared Error (MSE): 691855.3888144592
Mean Absolute Error (MAE): 455.77993200456956
R^2 Score: -0.23254348568216865


> R^2 상관계수가 낮게 나와 다른 모델링도 진행

# LinearRegression()

In [32]:
train = pd.read_csv("train_preprocessed.csv")
test = pd.read_csv("test_preprocessed.csv")

# 독립변수와 종속변수 분리
X = train.iloc[:, :-1]
y = train.iloc[:, -1]

# 학습용, 평가용 데이터 생성
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, test_size = 0.3, random_state = 2024)

In [35]:
# 선형 회귀 모델 생성
model = LinearRegression()

# 모델 학습
model.fit(X_train, y_train)

# 학습된 모델을 사용하여 예측
prediction = model.predict(X_test)
prediction

array([625.59924687,  58.90819438, -44.16256382, ..., -41.05879535,
       622.19323464, 667.81351854])

## 모델 평가

In [37]:
mse = mean_squared_error(y_test, prediction)
mae = mean_absolute_error(y_test, prediction)
r2 = r2_score(y_test, prediction)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R^2 Score:", r2)

Mean Squared Error (MSE): 453228.5170599618
Mean Absolute Error (MAE): 392.19687219276324
R^2 Score: 0.19257135918983703


## 성능 개선 - L2 규제 적용
- 독립변수간의 상호작용이 큰 경우라 생각하여 L2(Ridge) 규제를 적용하여 성능 개선 진행

In [51]:
ridge = Ridge()
params = {'alpha': [0.01, 0.1, 1.0, 10.0]}

grid = GridSearchCV(ridge, params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid.fit(X_train, y_train)

best_models = grid.best_estimator_
best_models

Ridge(alpha=10.0)

In [52]:
best_models.fit(X_train, y_train)

pred = best_models.predict(X_test)
pred

array([626.29685945,  59.367532  , -43.06593279, ..., -40.01142475,
       621.28676735, 643.20799169])

In [54]:
ridge_mse = mean_squared_error(y_test, pred)
ridge_mae = mean_absolute_error(y_test, pred)
ridge_r2 = r2_score(y_test, pred)

print("MSE:", ridge_mse)
print("MAE:", ridge_mae)
print("R^2 상관계수:", ridge_r2)

MSE: 453285.33443485625
MAE: 392.0847635440196
R^2 상관계수: 0.1924701387810176


> GridSearch를 통해 alpha 값을 조정한 모델의 상관계수가 더 떨어짐

# RandomForest

In [42]:
train = pd.read_csv("train_preprocessed.csv")
test = pd.read_csv("test_preprocessed.csv")

# 독립변수와 종속변수 분리
X = train.iloc[:, :-1]
y = train.iloc[:, -1]

# 학습용, 평가용 데이터 생성
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, test_size = 0.3, random_state = 2024)

In [43]:
model = RandomForestRegressor(n_estimators=100, random_state=42)  # 예시로 n_estimators=100 설정

model.fit(X_train, y_train)

pred = model.predict(X_test)

## 모델 평가

In [45]:
mse = mean_squared_error(y_test, pred)
mae = mean_absolute_error(y_test, pred)
r2 = r2_score(y_test, pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R^2 Score:", r2)

Mean Squared Error (MSE): 469253.2500437023
Mean Absolute Error (MAE): 370.8045785293261
R^2 Score: 0.16402322533378588


## 성능 개선 - GridSearch

In [ ]:
train = pd.read_csv("train_preprocessed.csv")
test = pd.read_csv("test_preprocessed.csv")

# 독립변수와 종속변수 분리
X = train.iloc[:, :-1]
y = train.iloc[:, -1]

# 학습용, 평가용 데이터 생성
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, test_size = 0.3, random_state = 2024)

In [46]:
from sklearn.model_selection import GridSearchCV

model = RandomForestRegressor(random_state=2024)

params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid = GridSearchCV(model, params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid.fit(X_train, y_train)

best_rf = grid.best_estimator_

In [47]:
best_rf

RandomForestRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=10,
                      random_state=2024)

In [49]:
# 모델 학습
best_rf.fit(X_train, y_train)

# 학습된 모델을 사용하여 예측
pred = best_rf.predict(X_test)

# 성능 평가
mse_rf = mean_squared_error(y_test, pred)
mae_rf = mean_absolute_error(y_test, pred)
r2_rf = r2_score(y_test, pred)

print(f'mse_rf = {mse_rf}, \nmae_rf = {mae_rf}, \nr2_rf = {r2_rf}')

mse_rf = 423221.22340451495, 
mae_rf = 349.74561635769425, 
r2_rf = 0.24602948774665823


# 결론

> **최종 선택 모델은 Linear Regression** 선형회귀 모델로, 모델의 설명력은 0.19로 낮지만 Decision Tree, RandomoForest 모델보다 나은 설명력을 보임